# DSCI100 Project Final Report
### Players Subscription Predictive Data Analysis

### Group 29:
- Ysabel Maria
- Sanjana Gopee - 59940676
- Simar Pandher - 14521397
- Olivia Kong

## Instructions for Final Report
- Title **(Done)**

- Introduction: Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report
clearly state the question you tried to answer with your project identify and fully describe the dataset that was used to answer the question **(Done)**

- Methods & Results: Describe the methods you used to perform your analysis from beginning to end that narrates the analysis code. **(In-progress)**

Your report should include code which:

1. Loads data 
2. Wrangles and cleans the data to the format necessary for the planned analysis
3. Performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
4. Creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
5. Performs the data analysis
6. Creates a visualization of the analysis 

Note: All figures should have a figure number and a legend

- Discussion:
1. Summarize what you found
2. Discuss whether this is what you expected to find?
3. Discuss what impact could such findings have?
4. Discuss what future questions could this lead to?

- References
You may include references if necessary, as long as they all have a consistent citation style.

In [2]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_validate



alt.data_transformers.enable('vegafusion')

set_config(transform_output="pandas")

### Introduction
This project is based on datasets that have been provided by a research group in Computer Science at UBC who obtained data regarding how individuals play video games. They did this through recording player actions in a MineCraft Server, and collecting data regarding information about each individual as well as how they play. This data has been condensed into the datasets of **Players** and **Sessions**. The question our group has chosen to answer is Question 3 provided in the criteria. 

**Question 3**: We would like to know something about our populations of users, in particular, we would like to have a good model of whether or not a player will continue contributing given past participation. 

## Data Description

## Players Data
  The players.csv filed is a data set containing information about the players in the game. There are 196 observations with data about the players such as their experience, whether they subscribe, their email, the number of hours played, their names, gender and age. These categories are split into the 9 variables (column names) below. 

  


**Players.csv:**

|     Variable     |  Type   |                    Description                            |
|------------------|---------|-----------------------------------------------------------|
| experience       | String  | The level of expertise that the player has in the game. Possible values are "Amateur", "Beginner", "Regular", "Pro", "Veteran". This is a categorical variable.              |
| subscribe        | Boolean | Whether or not the player has subscribed to the game. The variable can only take the value "TRUE" or "FALSE", indicating "yes"  or "no" to whether they are subscribed.  |
| hashedEmail      | String  | This is a string of letters and numbers to encrypt the email of the user. This is a unique identifier for each player.                        |
| played_hours     | Float   | The played hours indicates the number of hours spent playing the game approximated to one decimal place.                               |
| name             | String  | This is the name (first name) of the player. This is probably not a unique identifier since two people could coincidentally have the same name.                                             |
| gender           | String  | Gender is a categorical variable which has the following possible values: "Male", "Female", "Non-binary", "Prefer not to say", "Agender", "Two-spirited", "Other".                                        |
| age              | Integer | Player's age                                              |
| individualId     | N/A     | Individual ID of the player, values were not provided therefore the category is essentially useless in data analysis           |
| organizationName | N/A     | Organization name, values were not provided making this category useless just as the "IndividualID" column is.                  |

The final two columns are negligible, therefore will be dropped and the categories of use are reduced to 7 variables from 9.

## Sessions Data
The sessions.csv data set contains specific data about the playing sessions of the players in the game. There are 1535 observations in the sessions data set. It has variables like the players' email, start time, end time, original start time and original end time.

**Sessions.csv:**
|     Variable     |  Type   |                    Description                            |
|------------------|---------|-----------------------------------------------------------|
| hashedEmail      | String  | This is a string of letters and numbers to encrypt the email of the user. This is a unique identifier for each player.                       |
| start_time     | String   | This includes the date - in format DD/MM/YY - and the time the player started playing the game. The time is in 24-hour format.                               |
| end_time             | String  | This includes the date - in format DD/MM/YY - and the time the player stopped playing the game. The time is in 24-hour format.                                             |
| original_start_time           | Integer  | This variable is a 14-digit integer indicating the start time.                                           |
| original_end_time                | Integer | This variable is a 14-digit integer indicating the end time.                                              |

Unlike the players data, none of the variables are negligible as they all provide information about the playing sessions rather than being empty observations.


## Method

### Summary
Using data available about **Players** and **Sessions**, the goal of this project is to predict player retention, whether a player will continue playing the game based on the data from the two given dataframes. The column "subscribe" is the categorical variable we are trying to predict, and this is based on our assumption that a players subscribing to the game will continue to play it. The two varaibles chosen to predict this are:

1. Total number of hours played
2. Average session time per player

This will require a KNN classification model with the most optimal K value, and the two above quantitative predictor variables. The data will also be split into training and test sets to evaluate the classifer model's performance using accuracy, recall and precision. Through cross validation, an optimal k-value will be chosen. 

### Detailed Steps [INCOMPLETE - Still need to write more]
- **Data Wrangling**:

1. Converting dtype object to numeric
2. Groupby() and mean
3. Merging the two data sets based on hashedemail

- **Exploratory Analysis**: explain
- **Data Analysis**: explain
- **Visualization of Analysis**: explain

### Data Analysis
#### Players Data

In [7]:
# Load the dataset
players = pd.read_csv("data/players.csv")

In [8]:
# Drop the columns "individualID" and "organizationName" as they provide no observations, therefore are useless
players_tidy = players.drop(columns=['individualId', 'organizationName'])

#### Sessions Data

In [12]:
# Load the Dataset
sessions = pd.read_csv("data/sessions.csv")

In [13]:
# Convert all columns except for "hashedEmail" into datetime rather than str
sessions['start_time_final']=pd.to_datetime(sessions["start_time"], dayfirst=True)
sessions['end_time_final']=pd.to_datetime(sessions["end_time"], dayfirst=True)

In [14]:
# Calculate Session Length in days and time, adding a "session_length" column
sessions["session_length"] = sessions["end_time_final"]-sessions["start_time_final"]

In [15]:
#Grouping and Calculating Mean in Minutes
sessions_group=sessions.groupby('hashedEmail')["session_length"].mean().reset_index()
sessions_group["session_length_mins"] = sessions_group["session_length"].dt.total_seconds()/60

#### Merging Players Data and Sessions Data

In [16]:
# Merging datasets based on hashedEmail and dropping irrelevant columns (we are only intersted in hours played and whether or not each individual is subscribed)
merged=sessions_group.merge(players, on='hashedEmail')
merged.drop(columns=["session_length", "individualId", "organizationName", "name", "gender","age","experience"])

,hashedEmail,session_length_mins,subscribe,played_hours
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,53.000000,True,1.5
1,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,30.000000,False,0.4
2,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,11.000000,True,0.1
3,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,32.153846,True,5.6
4,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,35.000000,True,1.0
...,...,...,...,...
120,fc0224c81384770e93ca717f32713960144bf0b52ff676...,16.000000,True,0.2
121,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,80.000000,True,1.2
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,15.467742,True,56.1
123,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,9.000000,True,0.1


#### Solving Data Imbalance

In [17]:
# Use value_counts() to view the ratio of subscribed individuals to not subscribed individuals
merged["subscribe"].value_counts()

subscribe
True     93
False    32
Name: count, dtype: int64

This shows that there are way more people who are subscribed than not subscribed, which will cause issues in later classification. Therefore, we want to make the data more even by replicating the less common result (False) multiple times. 

In [18]:
subscribe_false = merged[merged["subscribe"] == False]
subscribe_true = merged[merged["subscribe"] == True]

subscribe_false_upsample = subscribe_false.sample(
n=subscribe_true.shape[0], replace=True)

upsampled_merged = pd.concat((subscribe_false_upsample, subscribe_true))
(upsampled_merged["subscribe"].value_counts())

subscribe
False    93
True     93
Name: count, dtype: int64

This output tells us that the values are now even after the less common result (False) has been replicated up to the same number of values for the more common result (True).

,hashedEmail,session_length,session_length_mins,experience,subscribe,played_hours,name,gender,age,individualId,organizationName
22,42eafe96ed5c1684e3b5cc614d1b01a117173d3ec6898a...,0 days 00:05:21.818181818,5.363636,Beginner,False,0.3,Ariana,Female,17,NaN,NaN
97,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0 days 00:05:00,5.000000,Amateur,False,0.0,Dylan,Prefer not to say,17,NaN,NaN
92,d3ca24e4d7fe8ffe2a821ebd3b841252950ca53e4d659a...,0 days 00:14:00,14.000000,Veteran,False,0.1,Casey,Male,18,NaN,NaN
87,c871393f513c68e313874ac9d192a0b8be7bf99a96bff0...,0 days 01:01:20,61.333333,Veteran,False,2.7,Sophia,Female,22,NaN,NaN
94,d46bd29a2ed08e3500bd8729085ef4b6f0ca65baf4c756...,0 days 01:50:00,110.000000,Pro,False,1.7,Asher,Male,17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
120,fc0224c81384770e93ca717f32713960144bf0b52ff676...,0 days 00:16:00,16.000000,Amateur,True,0.2,Gemna,Male,27,NaN,NaN
121,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,0 days 01:20:00,80.000000,Pro,True,1.2,Sakura,Male,17,NaN,NaN
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,0 days 00:15:28.064516129,15.467742,Amateur,True,56.1,Dana,Male,23,NaN,NaN
123,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,0 days 00:09:00,9.000000,Amateur,True,0.1,Fatima,Male,17,NaN,NaN
